In [1]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

warnings.filterwarnings("ignore")

## Execute a pipeline and setup a view generator

In [3]:
from freamon.adapters.mlinspect.provenance import from_py_file
view_generator = from_py_file('pipelines--mlinspect--amazon-reviews.py')

INFO:root:Patching sys.argv with ['eyes']
INFO:root:Executing instrumented user pipeline with mlinspect
INFO:root:Redirecting the pipeline's stdout to pipeline-output.txt
INFO:root:---RUNTIME: Instrumented execution took 16559.74507331848 ms
INFO:root:Registering test source 2 with columns: ['product_id', 'product_parent', 'product_title', 'category_id', 'mlinspect_lineage']
INFO:root:Registering test source 3 with columns: ['id', 'category', 'mlinspect_lineage']
INFO:root:Registering test source 1 with columns: ['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'mlinspect_lineage']
INFO:root:Registering test source 0 with columns: ['marketplace', 'customer_id', 'review_id', 'product_id', 'vine', 'third_party', 'review_headline', 'review_body', 'review_date', 'mlinspect_lineage']


In [8]:
view_generator.db.execute("SELECT * FROM _freamon_test_view").df()

,features,y_true,y_pred,prov_id_source_2,prov_id_source_3,prov_id_source_1,prov_id_source_0
0,[-1.00789514e-03 -1.15866240e+00 1.00000000e+...,0,0,0,46881,0,0
1,[0.02370451 0.76846785 1. ... 0.123091...,0,0,453,74300,0,0
2,[-0.07514512 0.76846785 1. ... 0. ...,1,1,2,51654,2,0
3,[-1.00789514e-03 -1.15866240e+00 1.00000000e+...,1,0,1202,28309,2,0
4,[-0.07514512 0.76846785 1. ... 0. ...,1,1,4853,42667,2,0
...,...,...,...,...,...,...,...
25950,[-0.0257203 -1.80103915 1. ... 0. ...,1,1,63663,61853,3600,0
25951,[-0.07514512 -1.1586624 1. ... -0.09...,1,1,63703,85708,5605,0
25952,[-0.0010079 0.76846785 1. ... 0. ...,0,0,63747,46227,3240,0
25953,[-0.07514512 -1.1586624 1. ... 0. ...,1,1,63775,21482,5608,0


In [9]:
view_generator.db.execute("SELECT * FROM _freamon_source_0_with_prov_view").df()

,marketplace,customer_id,review_id,product_id,vine,third_party,review_headline,review_body,review_date,prov_id_source_0
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,N,N,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31,0
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,N,Y,Five Stars,Awesome,2015-08-31,1
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,N,Y,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31,2
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,N,Y,Five Stars,Perfect,2015-08-31,3
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,N,Y,Five Stars,Awesome!,2015-08-31,4
...,...,...,...,...,...,...,...,...,...,...
246555,US,41754720,R19OFJV91M7D8X,B000YMR61A,N,N,"Easy to use, 1 comment 1 serious problem",I chose the deluxe version CD because of mortg...,2008-02-11,246555
246556,US,51669529,R1I6G894K5AGG5,B000YMR61A,N,N,Schedule C IS for business- figures it would ...,"Schedule C IS for business, so figures it wou...",2008-02-08,246556
246557,US,24731012,R17OE43FFEP81I,B000YMR5X4,N,N,Hassel to download,I wish that companies can test several scenari...,2008-02-05,246557
246558,US,16049580,R15MGDDK63B52Z,B000YMR61A,N,N,beware of vista,i just installed turbotax deluxe 2007. If you ...,2008-02-05,246558


In [10]:
view_generator.db.execute("SELECT * FROM _freamon_source_1_with_prov_view").df()

,review_id,star_rating,helpful_votes,total_votes,prov_id_source_1
0,R3MYFPQS43VI4U,1,0,1,0
1,R2X6C7MF5UNVHR,5,0,0,1
2,R1J2NC0YUY1Y43,3,26,26,2
3,R26ZROGA3BXGD8,5,0,0,3
4,R3I12FCDIY4YBT,5,0,1,4
...,...,...,...,...,...
99995,R31ZU225EBLRV7,5,0,0,99995
99996,R3BZ9VTST4H4MY,5,0,1,99996
99997,RC8AEW15R66CM,4,0,0,99997
99998,R1DSGCSRF1Z5BX,1,2,7,99998


In [11]:
view_generator.db.execute("SELECT * FROM _freamon_source_2_with_prov_view").df()

,product_id,product_parent,product_title,category_id,prov_id_source_2
0,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,0,0
1,B00F4CEHNK,341969535,Xbox Live Gift Card,0,1
2,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,0,2
3,B004RMK5QG,395682204,Playstation Plus Subscription,0,3
4,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,0,4
...,...,...,...,...,...
10927,B00194DS1Y,535540421,Better Homes and Gardens Landscaping and Deck ...,1,10927
10928,B002FL51XS,850943316,QuickBooks Pro 2009,1,10928
10929,B002Q35NVA,817509279,TomTom Mr. T Voice by NavTones [Online Map Cod...,1,10929
10930,B002FL51Y2,253109220,QuickBooks Premier Industry Editions 2009 [Dow...,1,10930


In [12]:
view_generator.db.execute("SELECT * FROM _freamon_source_3_with_prov_view").df()

,id,category,prov_id_source_3
0,0,Digital_Video_Games,0
1,1,Digital_Software,1


## Generate and materialize a view for data debugging

In [5]:
df = view_generator.test_view(
    sliceable_by=['category', 'marketplace'], 
    with_features=False, 
    with_y_true=True, 
    with_y_pred=True)

df

INFO:root:
SELECT fs3.category, fs0.marketplace, ftv.y_true, ftv.y_pred
FROM _freamon_test_view ftv
JOIN _freamon_source_0_with_prov_view fs0  ON fs0.prov_id_source_0 = ftv.prov_id_source_0
JOIN _freamon_source_3_with_prov_view fs3  ON fs3.prov_id_source_3 = ftv.prov_id_source_3
   
        


,category,marketplace,y_true,y_pred


## Compute group-wise confusion matrix (for fairness metrics) via an aggregation query

In [12]:
view_generator.query(
"""
SELECT 
    race=='White' AS privileged, 
    SUM(CAST((y_true=1 AND y_pred=1) AS INTEGER)) AS true_positive,
    SUM(CAST((y_true=1 AND y_pred=0) AS INTEGER)) AS false_negative,    
    SUM(CAST((y_true=0 AND y_pred=1) AS INTEGER)) AS false_positive,    
    SUM(CAST((y_true=0 AND y_pred=0) AS INTEGER)) AS true_negative,    
FROM df
GROUP BY privileged
"""
)

,privileged,true_positive,false_negative,false_positive,true_negative
0,True,168.0,214.0,106.0,1220.0
1,False,33.0,28.0,38.0,321.0


## Slicefinder via aggregation queries

In [14]:
view_generator.query(
"""
SELECT 
    race='White' AS white,
    sex='Male' AS male,
    AVG(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS avg_loss,
    VARIANCE(-(y_true * log(y_pred_proba) + (1 - y_true) * log(1.0 - y_pred_proba))) AS var_loss,    
    COUNT(*) as size
    
FROM (SELECT race, sex, y_true, IF(y_pred=0, 0.00001, 0.99999) AS y_pred_proba FROM df)
GROUP BY GROUPING SETS ((race='White', sex='Male'), (race='White'), (sex='Male'))
"""
)

,white,male,avg_loss,var_loss,size
0,True,False,0.616228,2.705607,641
1,False,True,0.949078,3.862504,216
2,True,True,1.129338,4.375369,1067
3,False,False,0.612749,2.701507,204
4,True,NaN,0.936772,3.808529,1708
5,False,NaN,0.785718,3.319121,420
6,NaN,False,0.615388,2.701417,845
7,NaN,True,1.098990,4.290499,1283
